In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import os
import math
import scipy.io
import collections

In [ ]:
data = pd.read_csv("datacgm2.csv")

In [ ]:
data

actBolusDelivered    dateMuBolus     dateNumber  numCGM
0                    0.0  737103.556169  737103.557257   118.0
1                    0.0  737103.556111  737103.553785   122.0
2                    0.0  737103.554722  737103.550312     NaN
3                    0.0  737103.554722  737103.546840     NaN
4                    0.0  737103.550382  737103.543368     NaN
...                  ...            ...            ...     ...
55338                NaN            NaN  736901.520069   311.0
55339                NaN            NaN  736901.516597   311.0
55340                NaN            NaN  736901.513125   309.0
55341                NaN            NaN  736901.509653   310.0
55342                NaN            NaN  736901.506181   314.0

[55343 rows x 4 columns]

In [ ]:
bolusDate = data['dateMuBolus'].dropna().tolist()
CGMDate = data['dateNumber']
bolousValue = data['actBolusDelivered'].dropna().tolist()
CGMValue = data['numCGM']

In [ ]:
print("CGM Dates", CGMDate[0:10])
print("Bolus Dates", bolusDate[0:10])

CGM Dates 0    2018-02-12 13:22:27.000003
1    2018-02-12 13:17:27.000001
2    2018-02-12 13:12:26.999998
3    2018-02-12 13:07:26.999996
4    2018-02-12 13:02:27.000004
5    2018-02-12 12:57:27.000002
6           2018-02-12 12:52:27
7    2018-02-12 12:47:26.999997
8    2018-02-12 12:42:26.999995
9    2018-02-12 12:37:27.000003
Name: dateNumber, dtype: object
Bolus Dates [datetime.datetime(2018, 2, 12, 13, 20, 53), datetime.datetime(2018, 2, 12, 13, 20, 47, 999997), datetime.datetime(2018, 2, 12, 13, 18, 48, 4), datetime.datetime(2018, 2, 12, 13, 18, 48, 4), datetime.datetime(2018, 2, 12, 13, 12, 32, 999996), datetime.datetime(2018, 2, 12, 13, 7, 35, 3), datetime.datetime(2018, 2, 12, 12, 57, 38, 2), datetime.datetime(2018, 2, 12, 12, 52, 39, 999999), datetime.datetime(2018, 2, 12, 12, 47, 39, 999997), datetime.datetime(2018, 2, 12, 12, 42, 41)]


In [ ]:
def nearest_timestamp(items, pivot):
  return min(items, key=lambda time: abs(time - pivot))

In [ ]:
assert len(CGMDate) == len(CGMValue)
assert len(bolusDate) == len(bolousValue)

date_to_cgm = dict()
for i in range(len(CGMDate)):
  date_to_cgm[CGMDate[i]] = CGMValue[i]

date_to_bolus = dict()
for i in range(len(bolusDate)):
  date_to_bolus[bolusDate[i]] = bolousValue[i]

In [ ]:
date_to_date = dict()
print(len(bolusDate))
for i in range(len(bolusDate)):
  nearest_date = nearest_timestamp(CGMDate, bolusDate[i])
  date_to_date[bolusDate[i]] = nearest_date

In [ ]:
df = pd.DataFrame(columns=["DateCGM", "Value CGM", "DateBolus", "Value Bolus"])

date_keys = list(date_to_date.keys())
CGMDate1, CGMValue1, bolusDate1, bolousValue1 = [], [], [], []

for i in range(len(date_keys)):
  CGMDate1.append(date_keys[i])
  CGMValue1.append(date_to_cgm[date_to_date[date_keys[i]]])
  bolusDate1.append(date_to_date[date_keys[i]])
  bolousValue1.append(date_to_bolus[date_keys[i]])

df['DateCGM'] = CGMDate1
df['Value CGM'] = CGMValue1
df['DateBolus'] = bolusDate1
df['Value Bolus'] = bolousValue1

df = df.dropna()
df.to_csv('FinalData.csv', index=False)
